In [1]:
%load_ext autoreload
%autoreload 2

Based on 



1.   https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379
2.   https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/
3.   https://huggingface.co/transformers/training.html#fine-tuning-with-keras




**problem statement:**


*   a developer has to inspect an **artifact X**
*   Within the artifact, only a portion of the text is relevant to **input task Y**
*   We ought to build a model that establishes relationships between **Y** and **sentences x ∈ X** 
*  The model must determine: **is x relevant to task Y**




<br>

___

*Example of a task and an annotated artifact:*

<br>

[<img src="https://i.imgur.com/Zj1317H.jpg">](https://i.imgur.com/Zj1317H.jpg)




* The coloured sentences are sentences annotated as relevant to the input task. 
* The warmer the color, the more annotators selected that portion of the text. 
* For simplicity, we process the data and used sentences 

<br>

___

*Ultimately, our data is a tuple representing:*


*   **text** = artifact sentence

*   **question** = task description

*   **source** = URL of the artifact

*   **category_index** = whether sentence is relevant [or not] for the input task

*   **weights** = number of participants who annotated sentence as relevant


<br>

___



In [2]:
# @title Import data as JSON
import itertools
import json
import logging
import os
import sys
import random
from pathlib import Path

from Levenshtein import ratio
from colorama import Fore, Style

logger = logging.getLogger()
logger.level = logging.DEBUG
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)

from ds_android import get_input_for_BERT

raw_data = get_input_for_BERT()

print('Sample entry from data:')
print(json.dumps(raw_data[0], indent=4, sort_keys=True))

39 129  https://developer.android.com/training/permissions/requesting
14 21  https://stackoverflow.com/questions/5233543
4 34  https://github.com/morenoh149/react-native-contacts/issues/516
27 63  https://guides.codepath.com/android/Understanding-App-Permissions
9 161  https://www.avg.com/en/signal/guide-to-android-app-permissions-how-to-use-them-smartly
9 15  https://developer.android.com/training/volley/request
14 65  https://stackoverflow.com/questions/28504524
20 59  https://medium.com/@JasonCromer/android-asynctask-http-request-tutorial-6b429d833e28
5 97  https://www.twilio.com/blog/5-ways-to-make-http-requests-in-java
4 12  https://stackoverflow.com/questions/33241952
6 33  https://github.com/realm/realm-java/issues/776
3 17  https://stackoverflow.com/questions/8712652
8 59  https://dzone.com/articles/android-rotate-and-scale
5 470  https://developer.android.com/reference/android/widget/TextView
7 11  https://stackoverflow.com/questions/19025301
8 95  https://docs.oracle.com/java

4 54  https://developer.android.com/training/gestures/scroll
4 16  https://stackoverflow.com/questions/39588322
20 196  https://developer.android.com/training/dependency-injection/dagger-android
6 44  https://stackoverflow.com/questions/57235136
24 121  https://guides.codepath.com/android/dependency-injection-with-dagger-2
Sample entry from data:
{
    "category_index": 1,
    "question": "Permission Denial when trying to access contacts in Android",
    "source": "https://developer.android.com/training/permissions/requesting",
    "text": "Every Android app runs in a limited-access sandbox.",
    "weights": 1
}


In [3]:
# @title DICT that will store fold results
# If there is a previous execution for the same configuration, we load it from disk

config_output = 'bert_ds_synthetic_best_config.json'

fold_results = dict()
if 'venn_diagram_set' not in fold_results:
    fold_results['venn_diagram_set'] = []
        
# if os.path.isfile(config_output):
#     logger.info(Fore.YELLOW + "Loading data from cache" + Style.RESET_ALL)
#     with open(config_output) as input_file:
#         fold_results = json.load(input_file)

In [4]:
# @title Set environment variables

import os
import contextlib
import tensorflow as tf
import os
import codecs
import numpy as np
import math
import json

import numpy as np
import pandas as pd

from collections import defaultdict, Counter
from tqdm import tqdm

USE_TPU = False
os.environ['TF_KERAS'] = '1'

# @title Initialize TPU Strategy
if USE_TPU:
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
    tf.contrib.distribute.initialize_tpu_system(resolver)
    strategy = tf.contrib.distribute.TPUStrategy(resolver)
    
from TFBertForTaskTextClassification import TFBertForTaskTextClassification
from TFBertForTaskTextClassification import TFBertForAndroidTaskTextClassification
from TFBertForTaskTextClassification import TFBertForSyntheticTaskTextClassification 

from metrics import MetricsAggregator

Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


# BERT

In [5]:
# Init the model
model = TFBertForSyntheticTaskTextClassification(model_id = 'bert-base-uncased')

# Configure filters. All other values are as default
model.target_output = 10
model.use_frame_filtering = True
model.match_frame_from_task = False
model.n_undersampling = 3 # this dataset is smaller, so we don't do as much undersampling
        
# Load tokenizer
model.tokenizer(cache_dir='/home/msarthur/scratch', local_files_only=True)

In [6]:
# model = TFBertForSequenceClassification.from_pretrained(model_id, cache_dir='/home/msarthur/scratch', local_files_only=True)

In [7]:
# @title 10-fold cross validation WIP
CORPUS = raw_data

all_tasks = sorted(list(set([d['question'] for d in raw_data])))
rseed = 20210343
random.seed(rseed)
random.shuffle(all_tasks)

from sklearn.model_selection import KFold


file_handler = logging.FileHandler('/home/msarthur/scratch/LOG-bert_ds_synthetic.ans')
file_handler.setLevel(logging.DEBUG)
logger.addHandler(file_handler)


n_splits = 10
kf = KFold(n_splits=n_splits, random_state=rseed)
np_tasks_arr = np.array(all_tasks)


# <------------------------------------------------------------------------- TRAIN
df_train, df_val, _, weights = model.get_train_val_test(
    CORPUS, [all_tasks[0]] # dummy test data
)

logger.info('-' * 10)
logger.info(Fore.RED + 'train'+ Style.RESET_ALL)
logger.info(str(df_train.category_index.value_counts()))
logger.info("")

logger.info(Fore.RED + 'weights'+ Style.RESET_ALL)
logger.info(str(weights))
logger.info('-' * 10)


# Encode X_train
train_encodings = model.encode(df_train)
train_labels = df_train['category_index'].tolist()

# Encode X_valid
val_encodings = model.encode(df_val)
val_labels = df_val['category_index'].tolist()


# https://huggingface.co/transformers/custom_datasets.html
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))


fine_tunned_keras_model = model.build(
    train_dataset, val_dataset, weights, 
    checkpoint_filepath='/home/msarthur/scratch/best_model', 
    cache_dir='/home/msarthur/scratch', 
    local_files_only=True
)


# <-------------------------------------------------------------------------------- FOLDS
idx_split = 0
for train_index, test_index in kf.split(np_tasks_arr):

    idx_split = str(idx_split)
    eval_fold = True
    # 10 runs per fold to avoid reporting peek results in a given fold
    if idx_split in fold_results and fold_results[idx_split]['run_cnt'] >= 5:
        logger.info(Fore.RED + f"Fold {idx_split} FULLY TESTED" + Style.RESET_ALL)
        eval_fold = False


    if eval_fold:
        model.metrics.reset_aggregators()

        test_tasks_lst = np_tasks_arr[test_index].tolist()

        logger.info("")
        logger.info(Fore.RED + f"Fold {idx_split}" + Style.RESET_ALL)
        logger.info('\n'.join(test_tasks_lst))

        # <------------------------------------------------------------------------- INPUT
        _, _, df_test, weights = model.get_train_val_test(
            CORPUS, test_tasks_lst
        )
        
        logger.info(Fore.RED + 'test'+ Style.RESET_ALL)
        logger.info(str(df_test.category_index.value_counts()))
        logger.info("")
        
        if model.match_frame_from_task:
            __frame_pairs = model.fn_frame_pairs.get_most_common_frame_relationships(df_train)
            model.sentence_task_frame_pairs = __frame_pairs

        # <------------------------------------------------------------------------- TEST
        logger.info("")
        logger.info(Fore.RED + f"Testing model" + Style.RESET_ALL)
        for source in df_test["source"].unique():
            df_source = df_test[df_test["source"] == source]   
            logger.info(source)
            
            model.test(source, df_source, fine_tunned_keras_model)
                        

        # <------------------------------------------------------------------------- METRICS   
        
        prediction_metrics, api_metrics, so_metrics, git_metrics, misc_metrics = model.get_evaluation_metrics()
        
        MetricsAggregator.add_idx_fold_results(
            idx_split, fold_results, prediction_metrics,
            api_metrics, so_metrics, git_metrics, misc_metrics
        )

        fold_results['venn_diagram_set'] += model.metrics.venn_diagram_set
        fold_results['venn_diagram_set'] = list(set(fold_results['venn_diagram_set']))


        _precision, _recall, _f1score = MetricsAggregator.avg_macro_metric_for(prediction_metrics)

        logger.info("")
        logger.info(Fore.YELLOW + "Model metrics" + Style.RESET_ALL)
        logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
        logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
        logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)


    idx_split = int(idx_split)
    idx_split += 1
    logger.info(f"next {idx_split}")


/home/msarthur/hface/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
100%|██████████| 7903/7903 [00:00<00:00, 2781070.94it/s]

----------
train
0    491
1    164
Name: category_index, dtype: int64

weights
{0: 1.0, 1: 3.0}
----------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x2b2c434c63d0>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expe

100%|██████████| 7903/7903 [00:00<00:00, 1846353.51it/s]

...

--------------------
Y
[0s] 31 [1s] 1
predicted
[0s] 22 [1s] 10
--------------------
Accuracy: 0.7188
macro_f1: 0.5060
Precision: 0.5500
Recall: 0.8548
F1: 0.5060
1 entries logged
https://stackoverflow.com/questions/2993085
https://developer.android.com/training/gestures/scale
https://stackoverflow.com/questions/2661536
--------------------
Y
[0s] 99 [1s] 1
predicted
[0s] 96 [1s] 4
--------------------
Accuracy: 0.9500
macro_f1: 0.4872
Precision: 0.4948
Recall: 0.4798
F1: 0.4872
0 entries logged
https://stackoverflow.com/questions/10630373
--------------------
Y
[0s] 29 [1s] 3
predicted
[0s] 26 [1s] 6
--------------------
Accuracy: 0.8438
macro_f1: 0.6768
Precision: 0.6474
Recall: 0.7644
F1: 0.6768
2 entries logged
https://www.twilio.com/blog/asynchronous-api-requests-java-completablefutures
--------------------
Y
[0s] 48 [1s] 2
predicted
[0s] 40 [1s] 10
--------------------
Accuracy: 0.8000
macro_f1: 0.5265
Precision: 0.5375
Recall: 0.6562
F1: 0.5265
1 entries logged

Model metrics
pr

/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 7903/7903 [00:00<00:00, 2223178.04it/s]

test
0    553
1     25
Name: category_index, dtype: int64


Testing model
https://developer.android.com/training/data-storage/sqlite


--------------------
Y
[0s] 67 [1s] 2
predicted
[0s] 59 [1s] 10
--------------------
Accuracy: 0.8261
macro_f1: 0.4524
Precision: 0.4831
Recall: 0.4254
F1: 0.4524
0 entries logged
https://stackoverflow.com/questions/4015026
--------------------
Y
[0s] 26 [1s] 9
predicted
[0s] 27 [1s] 8
--------------------
Accuracy: 0.6857
macro_f1: 0.5727
Precision: 0.5764
Recall: 0.5705
F1: 0.5727
3 entries logged
https://developer.android.com/training/dependency-injection/hilt-android
--------------------
Y
[0s] 141 [1s] 4
predicted
[0s] 124 [1s] 21
--------------------
Accuracy: 0.8552
macro_f1: 0.5404
Precision: 0.5396
Recall: 0.6826
F1: 0.5404
2 entries logged
https://prog.world/a-practical-guide-to-using-hilt-with-kotlin
--------------------
Y
[0s] 45 [1s] 3
predicted
[0s] 38 [1s] 10
--------------------
Accuracy: 0.7708
macro_f1: 0.5107
Precision: 0.5237
Recall: 0.5667
F1: 0.5107
1 entries logged
https://stackoverflow.com/questions/8184492
--------------------
Y
[0s] 50 [1s] 3
predicted
[0s] 43

/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
# for source in df_test["source"].unique():
#     df_source = df_test[df_test["source"] == source]   
#     logger.info(source)
#     test_model(source, df_source, model, tokenizer, pos_filter=True)
    

In [9]:
__precision, __recall, __fscore = MetricsAggregator.get_full_exec_results(fold_results)

logger.info("\n")
logger.info(Fore.RED + "AGGREGATED METRICS" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)



AGGREGATED METRICS

precision: 0.520
recall:    0.548
f1-score:  0.504


In [10]:
## Stack Overflow results

In [11]:
__precision, __recall, __fscore = MetricsAggregator.get_full_exec_results(fold_results, result_type="so")

logger.info("\n")
logger.info(Fore.RED + "Stack Overflow metrics" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)



Stack Overflow metrics

precision: 0.565
recall:    0.611
f1-score:  0.554


In [12]:
## Github issues results

In [13]:
__precision, __recall, __fscore = MetricsAggregator.get_full_exec_results(fold_results, result_type="git")

logger.info("\n")
logger.info(Fore.RED + "Github issues metrics" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)



Github issues metrics

precision: 0.525
recall:    0.581
f1-score:  0.503


In [14]:
#@title Sample prediction outputs for API sources

logger.info(Fore.RED + "API" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='api', n_samples=8)

API

The gravity is not working on the TextView in some situation.
https://developer.android.com/reference/android/widget/TextView


Android PDF Rendering
https://developer.android.com/reference/android/graphics/pdf/PdfRenderer

[w=0][y=1][p=0.9049] It is guaranteed, however, that the thread that invokes finalize will not be holding any user-visible synchronization locks when finalize is invoked.

[w=0][y=1][p=0.8953] Subclasses of Object may override this definition.

[w=0][y=1][p=0.8949] For example, the finalize method for an object that represents an input/output connection might perform explicit I/O transactions to break the connection before the object is permanently discarded.

[w=0][y=1][p=0.4253] Gets the number of pages in the document.

[w=3][y=1][p=0.3711] If document page size is greater than the printed media size the content should be anchored to the upper left corner of the page for left-to-right locales and top right corner for right-to-left locales.

[w=3][y=1][p=0.32


[w=0][y=1][p=0.2290] Note: In the above example, the prepareVideoRecorder ( ) method refers to the example code shown in Configuring MediaRecorder.

[w=0][y=1][p=0.2167] Location Permission - If your application tags images with GPS location information, you must request the ACCESS_FINE_LOCATION permission.

[w=0][y=1][p=0.2041] This method is available in Android 2.2 ( API Level 8 ), for equivalent calls in earlier API versions, see Saving Shared Files.

--------------------

Explanation of the getView() method of an ArrayAdapter
https://developer.android.com/reference/android/widget/ArrayAdapter



In [15]:
#@title Sample prediction outputs for GIT sources

logger.info(Fore.RED + "GIT" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='git', n_samples=8)

GIT

SeekTo Position of cutted song not working
https://github.com/google/ExoPlayer/issues/8387

[w=1][y=1][p=0.8937] If you have a custom DataSource implementation then you'll need to share it so that we can take a look at what might be wrong.

[w=2][y=1][p=0.8818] The dataSpec argument is not defining the entire media.

[w=1][y=1][p=0.8563] It's not clear how you are actually applying the offset and length in or before FileDataSource.

[w=0][y=1][p=0.4815] One thing to note about this issue is that media can safely play without calling seek

[w=1][y=1][p=0.2875] It's defining the part of the media that the caller wants to read for this particular call to open.

[w=0][y=1][p=0.2149] Android device: Pixel 4 XL

[w=0][y=1][p=0.1437] The reason why dataSpec.position needs to be included is already explained as clearly as I'm able to explain it in my response above.

[w=0][y=1][p=0.1237] locked and limited conversation to collaborators

[w=0][y=1][p=0.1236] The Audio dies after you call s

In [22]:
#@title Sample prediction outputs for SO sources

logger.info(Fore.RED + "SO" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='so', n_samples=8)

SO

How to Integrate reCAPTCHA 2.0 in Android
https://stackoverflow.com/questions/27297067

[w=1][y=1][p=0.8142] So you have to validate this token.

[w=2][y=1][p=0.7743] Validating the response is really easy.

[w=3][y=1][p=0.3210] That means, that the widget will take care of asking questions, validating responses all the way till it determines that a user is actually a human, only then you get a g-recaptcha-response value.

[w=1][y=1][p=0.2119] Edit: It's actually a POST, as per LINK.

[w=1][y=1][p=0.1904] You will get a JSON Response with a success field.

[w=3][y=1][p=0.1721] Just make a GET Request to

[w=0][y=1][p=0.1526] Returns the API response in a JsonObject.

[w=0][y=1][p=0.1460] This code put at head section on call get_action -LRB- this -RRB- method form button:

--------------------

How to record phone calls in Android
https://stackoverflow.com/questions/6688444

[w=0][y=1][p=0.5336] after that, you can easily start recording anywhere you want

[w=0][y=1][p=0.4235] Firs

In [21]:
#@title Sample prediction outputs for MISC sources

logger.info(Fore.RED + "MISC" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='misc', n_samples=8)

MISC


In [18]:
logger.info(Fore.RED + f"{len(fold_results['venn_diagram_set'])} entries VENN SET" + Style.RESET_ALL)
for _t in fold_results['venn_diagram_set']:
    logger.info(_t)

77 entries VENN SET
then, in your code you detect the ENABLE_CRASHLYTICS flag as follows:

it is used for the Android Music Remote control even if the App is in Lock mode.
It sounds like you may be using the support library ?
I have no experience with Gradle, but it seems like you need to do something like this.
There is a WAKE_LOCK permission in your Manifest.xml.
Whether you need the authentication headers or not will decide which instance to inject.
If the device is running Android 6.0 or higher, and your app's target SDK is 23 or higher: The app has to list the permissions in the manifest, and it must request each dangerous permission it needs while the app is running.
However, there are a few cases where your app might need to implement its own Back behavior in order to provide the best possible user experience.
The TextView being in wrap_content this does nothing, as the TextView is exactly the size of the text.
An example of polymorphic deserialization is if you want your JSON o

In [19]:
fold_results

{'venn_diagram_set': ['then, in your code you detect the ENABLE_CRASHLYTICS flag as follows:',
  '',
  'it is used for the Android Music Remote control even if the App is in Lock mode.',
  'It sounds like you may be using the support library ?',
  'I have no experience with Gradle, but it seems like you need to do something like this.',
  'There is a WAKE_LOCK permission in your Manifest.xml.',
  'Whether you need the authentication headers or not will decide which instance to inject.',
  "If the device is running Android 6.0 or higher, and your app's target SDK is 23 or higher: The app has to list the permissions in the manifest, and it must request each dangerous permission it needs while the app is running.",
  'However, there are a few cases where your app might need to implement its own Back behavior in order to provide the best possible user experience.',
  'The TextView being in wrap_content this does nothing, as the TextView is exactly the size of the text.',
  'An example of p